# [4.4 Consistency Algorithms](http://artint.info/2e/html/ArtInt2e.Ch4.S4.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=57) (page 57)

## About
A more advanced technique for solving Constraint Satisfaction Problems (CSPs) is to use __arc consistency__ to prune domains first. In the best case, this will find a unique solution; otherwise, we can perform domain splitting or use search on the simpler problem.

## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter*. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

In [3]:
from aispace2.jupyter.csp import Displayable, visualize


class Con_solver(Displayable):
    def __init__(self, csp, **kwargs):
        """a CSP solver that uses arc consistency
        * csp is the CSP to be solved
        * kwargs is the keyword arguments for Displayable superclass
        """
        self.csp = csp
        super().__init__(**kwargs)    # Or Displayable.__init__(self,**kwargs)

    @visualize
    def make_arc_consistent(self, orig_domains=None, to_do=None):
        """Makes this CSP arc-consistent using generalized arc consistency
        orig_domains is the original domains
        to_do is a set of (variable,constraint) pairs
        returns the reduced domains
        """
        if orig_domains is None:
            orig_domains = self.csp.domains
        if to_do is None:
            to_do = {(var, const) for const in self.csp.constraints
                     for var in const.scope}
        else:
            to_do = to_do.copy()  # use a copy of to_do
        domains = orig_domains.copy()
        self.display(2, "Performing AC with domains", domains)
        while to_do:
            var, const = self.select_arc(to_do)
            self.display(3, "Processing arc (", var, ",", const, ")")
            other_vars = [ov for ov in const.scope if ov is not var]
            new_domain = {val for val in domains[var]
                          if self.any_holds(domains, const, {var: val}, other_vars)}
            if new_domain != domains[var]:
                self.display(4, "Arc: (", var, ",", const, ") is inconsistent")
                self.display(3, "Domain pruned", "dom(", var, ") =", new_domain,
                             " due to ", const)
                domains[var] = new_domain
                add_to_do = self.new_to_do(var, const) - to_do
                to_do |= add_to_do      # set union
                self.display(
                    3, "  adding", add_to_do if add_to_do else "nothing", "to to_do.")
            self.display(4, "Arc: (", var, ",", const, ") now consistent")
        self.display(2, "AC done. Reduced domains", domains)
        return domains

    def new_to_do(self, var, const):
        """returns new elements to be added to to_do after assigning
        variable var in constraint const.
        """
        return {(nvar, nconst) for nconst in self.csp.var_to_const[var]
                if nconst != const
                for nvar in nconst.scope
                if nvar != var}

    def select_arc(self, to_do):
        """Selects the arc to be taken from to_do .
        * to_do is a set of arcs, where an arc is a (variable,constraint) pair
        the element selected must be removed from to_do.
        """
        return self.visualizer.wait_for_arc_selection(to_do)

    def any_holds(self, domains, const, env, other_vars, ind=0):
        """returns True if Constraint const holds for an assignment
        that extends env with the variables in other_vars[ind:]
        env is a dictionary
        Warning: this has side effects and changes the elements of env
        """
        if ind == len(other_vars):
            return const.holds(env)
        else:
            var = other_vars[ind]
            for val in domains[var]:
                # env = dict_union(env,{var:val})  # no side effects!
                env[var] = val
                if self.any_holds(domains, const, env, other_vars, ind + 1):
                    return True
            return False

    @visualize
    def solve_one(self, domains=None, to_do=None):
        """return a solution to the current CSP or False if there are no solutions
        to_do is the list of arcs to check
        """
        if domains is None:
            domains = self.csp.domains
        new_domains = self.make_arc_consistent(domains, to_do)
        if any(len(new_domains[var]) == 0 for var in domains):
            return False
        elif all(len(new_domains[var]) == 1 for var in domains):
            self.display(2, "solution:", {var: select(
                new_domains[var]) for var in new_domains})
            return {var: select(new_domains[var]) for var in domains}
        else:
            var = self.split_var(x for x in self.csp.variables if len(
                new_domains[x]) > 1)
            if var:
                dom1, dom2 = self.partition_domain(new_domains[var])
                self.display(3, "...splitting", var, "into", dom1, "and", dom2)
                new_doms1 = copy_with_assign(new_domains, var, dom1)
                new_doms2 = copy_with_assign(new_domains, var, dom2)
                to_do = self.new_to_do(var, None)
                return self.solve_one(new_doms1, to_do) or self.solve_one(new_doms2, to_do)

    def split_var(self, iter_vars):
        return self.visualizer.wait_for_var_selection(iter_vars)

    def partition_domain(self, dom):
        """partitions domain dom into two.
        """
        return self.visualizer.choose_domain_partition(dom)


def copy_with_assign(domains, var=None, new_domain={True, False}):
    """create a copy of the domains with an assignment var=new_domain
    if var==None then it is just a copy.
    """
    newdoms = domains.copy()
    if var is not None:
        newdoms[var] = new_domain
    return newdoms


def select(iterable):
    """select an element of iterable. Returns None if there is no such element.

    This implementation just picks the first element.
    For many of the uses, which element is selected does not affect correctness, 
    but may affect efficiency.
    """
    for e in iterable:
        return e  # returns first element found

You can click on an arc to run `make_arc_consistent()` on that selection, or choose "Fine Step", "Step", or "Auto Arc-Consistency" to randomly select an arc to make consistent. If you run `solve_one()`, after arc consistency is finished, you can click on a variable with more than one value in its domain to split it. It will then prompt you to choose the domain of the first split.

In [4]:
from aipython.cspExamples import simple_csp1, simple_csp2, extended_csp, crossword1, crossword2, crossword2d
from IPython.display import display

con_solver = Con_solver(crossword1)

# Visualization options
con_solver.visualizer.line_width = 10
con_solver.visualizer.sleep_time = 0.1
con_solver.visualizer.text_size = 15

display(con_solver)

# Call the function to execute in our visualization
con_solver.solve_one()

Con_solver(graph=CSP({'one_across': {'ant', 'bus', 'big', 'has', 'car'}, 'one_down': {'book', 'buys', 'lane', 'year', 'hold'}, 'two_down': {'syntax', 'symbol', 'search', 'ginger'}, 'three_across': {'book', 'buys', 'year', 'hold', 'land'}, 'four_across': {'ant', 'bus', 'big', 'has', 'car'}}, ["meet_at(0,0)('one_across', 'one_down')", "meet_at(2,0)('one_across', 'two_down')", "meet_at(2,2)('three_across', 'two_down')", "meet_at(0,2)('three_across', 'one_down')", "meet_at(0,4)('four_across', 'two_down')"]), line_width=10.0, text_size=15)